In [5]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly
import plotly.express as px
import geopandas as gpd
import contextily as ctx
import rioxarray
import urbanpy as up
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tqdm.notebook import tqdm
from geocube.api.core import make_geocube

In [6]:
tqdm.pandas()

In [7]:
# Read the combined cells with access variables
gdf_cells_access = gpd.read_parquet("outputs/celdas_combined_access_v2.parquet")
gdf_cells_access_br = gpd.read_parquet("outputs/celdas_combined_bra_access_v2.parquet")

In [8]:
gdf_cells_access.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 18275 entries, 0 to 18274
Data columns (total 34 columns):
 #   Column                                       Non-Null Count  Dtype   
---  ------                                       --------------  -----   
 0   cell_id                                      18275 non-null  object  
 1   smod                                         18275 non-null  object  
 2   pop_1975                                     18275 non-null  float64 
 3   pop_1980                                     18275 non-null  float64 
 4   pop_1985                                     18275 non-null  float64 
 5   pop_1990                                     18275 non-null  float64 
 6   pop_1995                                     18275 non-null  float64 
 7   pop_2000                                     18275 non-null  float64 
 8   pop_2005                                     18275 non-null  float64 
 9   pop_2010                                     18275 no

In [9]:
gdf_cells_access.head()

,cell_id,smod,pop_1975,pop_1980,pop_1985,pop_1990,pop_1995,pop_2000,pop_2005,pop_2010,...,unique_id,lat_primary_school,lon_primary_school,nearest_secondary_schools_ix,distance_to_nearest_secondary_schools,duration_to_nearest_secondary_schools,duration_to_nearest_secondary_schools_label,unique_id_secondary_school,lat_secondary_school,lon_secondary_school
0,13887,urban_cluster,104.825494,223.638531,365.367907,532.631180,756.331140,1045.053330,1385.446574,1766.728589,...,boliviaID0013532,-10.947500,-69.564774,2804,1.6730,20.075000,15-30,boliviaID0013532,-10.947500,-69.564774
1,13956,rural,27.399182,67.594099,122.557618,197.463636,290.430222,397.374168,536.507827,717.496279,...,boliviaID0013394,-11.134600,-69.215324,2802,1.0347,12.415000,0-15,boliviaID0013394,-11.134600,-69.215324
2,13993,rural,1.727404,5.890069,19.501926,41.905899,93.348261,246.334796,451.661259,730.711286,...,boliviaID0013337,-11.280300,-69.069313,2790,0.2030,2.436667,0-15,boliviaID0013337,-11.280300,-69.069313
3,13998,rural,10.727191,22.730851,37.306889,54.579556,88.652168,146.463163,221.419988,318.644286,...,boliviaID0013370,-11.185100,-69.028412,2797,0.5786,6.943333,0-15,boliviaID0013370,-11.185100,-69.028412
4,14081,rural,41.429706,93.693430,159.420188,241.236232,339.788430,458.116727,644.444557,928.415508,...,boliviaID0013188,-11.879609,-68.986047,2773,0.5990,7.186667,0-15,boliviaID0013188,-11.879609,-68.986047


In [10]:
gdf_cells_access_br.head()

,index,cell_id,smod,pop_1975,pop_1980,pop_1985,pop_1990,pop_1995,pop_2000,pop_2005,...,unique_id,lat_primary_school,lon_primary_school,nearest_secondary_schools_ix,distance_to_nearest_secondary_schools,duration_to_nearest_secondary_schools,duration_to_nearest_secondary_schools_label,unique_id_secondary_school,lat_secondary_school,lon_secondary_school
0,0,10990,rural,33.774789,73.684276,122.963361,182.253686,255.843079,375.113321,516.263954,...,brasil12001821,-7.525277,-73.005554,494,82.5490,990.581667,>120,brasil12001821,-7.525277,-73.005554
1,1,11340,rural,40.787618,86.003471,143.616033,213.308203,315.076869,454.252589,625.666751,...,brasil12001821,-7.525277,-73.005554,494,80.5647,966.770000,>120,brasil12001821,-7.525277,-73.005554
2,2,11420,rural,66.769884,137.195748,217.874020,308.875040,413.820839,538.151761,692.404688,...,brasil12001821,-7.525277,-73.005554,494,133.4141,1600.955000,>120,brasil12001821,-7.525277,-73.005554
3,3,11429,rural,57.325188,119.504881,191.428826,272.743450,365.787873,475.210036,610.950871,...,brasil12001821,-7.525277,-73.005554,494,133.4141,1600.955000,>120,brasil12001821,-7.525277,-73.005554
4,4,11531,rural,27.586158,60.270482,105.306524,166.063003,236.071840,316.338729,413.987196,...,brasil12001821,-7.525277,-73.005554,494,1.0877,13.056667,0-15,brasil12001821,-7.525277,-73.005554


In [11]:
gdf_cells_access_br = gdf_cells_access_br.drop("index", axis=1)

In [12]:
gdf_cells_access.columns

Index(['cell_id', 'smod', 'pop_1975', 'pop_1980', 'pop_1985', 'pop_1990',
       'pop_1995', 'pop_2000', 'pop_2005', 'pop_2010', 'pop_2015', 'pop_2020',
       'polygon_id', 'geometry', 'index_countries', 'ADM0_EN', 'ADM0_PCODE',
       'code', 'lon', 'lat', 'nearest_primary_schools_ix',
       'distance_to_nearest_primary_schools',
       'duration_to_nearest_primary_schools',
       'duration_to_nearest_primary_schools_label', 'unique_id',
       'lat_primary_school', 'lon_primary_school',
       'nearest_secondary_schools_ix', 'distance_to_nearest_secondary_schools',
       'duration_to_nearest_secondary_schools',
       'duration_to_nearest_secondary_schools_label',
       'unique_id_secondary_school', 'lat_secondary_school',
       'lon_secondary_school'],
      dtype='object')

In [13]:
filter_cols = [
    # Cell variables
    "cell_id",
    "polygon_id",
    "smod",  # urbanization degree
    "code",  # country
    "lon", "lat",  # cell centroid
    "geometry",  # cell polygon
    # Accessibility to primary schools
    "nearest_primary_schools_ix",
    "distance_to_nearest_primary_schools",
    "duration_to_nearest_primary_schools",
    "duration_to_nearest_primary_schools_label",
    "unique_id",
    "lat_primary_school",
    "lon_primary_school",
    # Acessibility to secondary schools
    "nearest_secondary_schools_ix",
    "distance_to_nearest_secondary_schools",
    "duration_to_nearest_secondary_schools",
    "duration_to_nearest_secondary_schools_label",
    "unique_id_secondary_school",
    "lat_secondary_school",
    "lon_secondary_school",
]

In [14]:
gdf_cells_access_concat = pd.concat(
    [gdf_cells_access[filter_cols], gdf_cells_access_br[filter_cols]], ignore_index=True
)

In [32]:
gdf_cells_access_concat.columns

Index(['cell_id', 'polygon_id', 'smod', 'code', 'lon', 'lat', 'geometry',
       'nearest_primary_schools_ix', 'distance_to_nearest_primary_schools',
       'duration_to_nearest_primary_schools',
       'duration_to_nearest_primary_schools_label', 'unique_id',
       'lat_primary_school', 'lon_primary_school',
       'nearest_secondary_schools_ix', 'distance_to_nearest_secondary_schools',
       'duration_to_nearest_secondary_schools',
       'duration_to_nearest_secondary_schools_label',
       'unique_id_secondary_school', 'lat_secondary_school',
       'lon_secondary_school'],
      dtype='object')

In [15]:
# Read the combined cells with the worldpop variables
gdf_cells_pop = gpd.read_parquet("outputs/celdas_combined_pop.parquet")
gdf_cells_pop_bra = gpd.read_parquet(
    "outputs/brazil_worldpop_school_age_celdas.parquet"
)

In [16]:
gdf_cells_pop.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 18275 entries, 0 to 22586
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   cell_id                   18275 non-null  int32   
 1   smod                      18275 non-null  object  
 2   pop_1975                  18275 non-null  float64 
 3   pop_1980                  18275 non-null  float64 
 4   pop_1985                  18275 non-null  float64 
 5   pop_1990                  18275 non-null  float64 
 6   pop_1995                  18275 non-null  float64 
 7   pop_2000                  18275 non-null  float64 
 8   pop_2005                  18275 non-null  float64 
 9   pop_2010                  18275 non-null  float64 
 10  pop_2015                  18275 non-null  float64 
 11  pop_2020                  18275 non-null  float64 
 12  polygon_id                18275 non-null  object  
 13  geometry                  18275 non-null  g

In [17]:
gdf_cells_pop.head()

,cell_id,smod,pop_1975,pop_1980,pop_1985,pop_1990,pop_1995,pop_2000,pop_2005,pop_2010,...,pop_2020_m_10,pop_2020_f_10,pop_2020_m_15,pop_2020_f_15,country,pop_m,pop_f,pop_total,pop_primary_school_age,pop_secondary_school_age
0,1,urban_cluster,773.876615,1497.169295,2214.783282,2936.382639,3759.680520,4684.930881,5639.160211,6578.468722,...,16.853684,15.952408,12.418421,11.182146,per,46.182592,43.726013,89.908605,33.501947,56.406659
1,2,rural,127.349266,262.085135,409.920769,567.952579,738.324368,919.774243,1098.632136,1267.729462,...,5.357383,5.070888,3.947519,3.554536,per,14.680340,13.899452,28.579792,10.649466,17.930326
2,3,rural,128.605609,273.402513,435.992956,620.728641,817.119036,1023.953282,1225.916293,1414.747791,...,4.599114,4.353169,3.388798,3.051437,per,12.602526,11.932163,24.534689,9.142171,15.392518
3,4,rural,127.938058,263.618495,410.432105,568.264752,735.777852,912.002276,1085.084391,1247.965492,...,5.083956,4.812084,3.746048,3.373122,per,13.931095,13.190062,27.121157,10.105947,17.015211
4,5,urban_cluster,176.856075,356.591034,548.408754,750.600224,1013.627816,1348.696712,1691.794096,2031.490143,...,8.575487,8.116899,6.318737,5.689696,per,23.498613,22.248657,45.747271,17.046451,28.700820


In [18]:
gdf_cells_pop_bra.head()

,cell_id,smod,geometry,code,pop_2020_m_5,pop_2020_f_5,pop_2020_m_10,pop_2020_f_10,pop_2020_m_15,pop_2020_f_15,pop_m,pop_f,pop_total,pop_primary_school_age,pop_secondary_school_age
0,10990,rural,"POLYGON ((-73.44861 -7.43248, -73.44778 -7.424...",bra,0.329185,0.287719,0.279947,0.268754,0.259563,0.239487,0.868696,0.795961,1.664657,0.616905,1.047752
1,11340,rural,"POLYGON ((-73.21709 -7.42437, -73.21626 -7.416...",bra,0.560032,0.489488,0.476265,0.457222,0.441587,0.407432,1.477884,1.354141,2.832025,1.049519,1.782506
2,11420,rural,"POLYGON ((-72.97373 -5.73784, -72.97309 -5.729...",bra,16.507923,16.430574,13.899233,13.425953,11.649813,12.238817,42.056969,42.095345,84.152313,32.938498,51.213816
3,11429,rural,"POLYGON ((-72.96436 -5.74594, -72.96372 -5.737...",bra,3.606850,3.589950,3.036872,2.933464,2.545391,2.674084,9.189113,9.197498,18.386611,7.196800,11.189811
4,11531,rural,"POLYGON ((-72.99814 -7.53797, -72.99729 -7.529...",bra,5.784639,5.055977,4.919402,4.722705,4.561205,4.208414,15.265246,13.987096,29.252343,10.840617,18.411726


In [19]:
gdf_cells_pop.columns

Index(['cell_id', 'smod', 'pop_1975', 'pop_1980', 'pop_1985', 'pop_1990',
       'pop_1995', 'pop_2000', 'pop_2005', 'pop_2010', 'pop_2015', 'pop_2020',
       'polygon_id', 'geometry', 'pop_2020_m_5', 'pop_2020_f_5',
       'pop_2020_m_10', 'pop_2020_f_10', 'pop_2020_m_15', 'pop_2020_f_15',
       'country', 'pop_m', 'pop_f', 'pop_total', 'pop_primary_school_age',
       'pop_secondary_school_age'],
      dtype='object')

In [20]:
gdf_cells_pop_bra["polygon_id"] = range(1000000, 1000000 + len(gdf_cells_pop_bra))

In [21]:
gdf_cells_pop_bra = gdf_cells_pop_bra.rename({"code": "country"}, axis=1)

In [22]:
gdf_cells_pop_bra.columns

Index(['cell_id', 'smod', 'geometry', 'country', 'pop_2020_m_5',
       'pop_2020_f_5', 'pop_2020_m_10', 'pop_2020_f_10', 'pop_2020_m_15',
       'pop_2020_f_15', 'pop_m', 'pop_f', 'pop_total',
       'pop_primary_school_age', 'pop_secondary_school_age', 'polygon_id'],
      dtype='object')

In [23]:
gdf_cells_pop_concat = pd.concat(
    [gdf_cells_pop[gdf_cells_pop_bra.columns], gdf_cells_pop_bra], ignore_index=True
)

In [33]:
index_col = ["cell_id"]
common_cols = ["smod", "polygon_id", "geometry", "country"]
access_cols = [
    "lat", "lon", 
    "nearest_primary_schools_ix",
    "distance_to_nearest_primary_schools",
    "duration_to_nearest_primary_schools",
    "duration_to_nearest_primary_schools_label",
    "unique_id",
    "lat_primary_school",
    "lon_primary_school",
    "nearest_secondary_schools_ix",
    "distance_to_nearest_secondary_schools",
    "duration_to_nearest_secondary_schools",
    "duration_to_nearest_secondary_schools_label",
    "unique_id_secondary_school",
    "lat_secondary_school",
    "lon_secondary_school",
]
pop_cols = [
    "pop_2020_m_5",
    "pop_2020_f_5",
    "pop_2020_m_10",
    "pop_2020_f_10",
    "pop_2020_m_15",
    "pop_2020_f_15",
    "pop_m",
    "pop_f",
    "pop_total",
    "pop_primary_school_age",
    "pop_secondary_school_age",
]

In [34]:
gdf_cells_pop_concat

,cell_id,smod,geometry,country,pop_2020_m_5,pop_2020_f_5,pop_2020_m_10,pop_2020_f_10,pop_2020_m_15,pop_2020_f_15,pop_m,pop_f,pop_total,pop_primary_school_age,pop_secondary_school_age,polygon_id
0,1,urban_cluster,"POLYGON ((-79.52572 -5.57578, -79.52504 -5.567...",per,16.910487,16.591460,16.853684,15.952408,12.418421,11.182146,46.182592,43.726013,89.908605,33.501947,56.406659,02559-1-1
1,2,rural,"POLYGON ((-79.50584 -5.33273, -79.5052 -5.3246...",per,5.375439,5.274028,5.357383,5.070888,3.947519,3.554536,14.680340,13.899452,28.579792,10.649466,17.930326,07918-1-1
2,3,rural,"POLYGON ((-79.52322 -5.66491, -79.52253 -5.656...",per,4.614614,4.527557,4.599114,4.353169,3.388798,3.051437,12.602526,11.932163,24.534689,9.142171,15.392518,06180-1-1
3,4,rural,"POLYGON ((-79.52253 -5.65681, -79.52185 -5.648...",per,5.101091,5.004856,5.083956,4.812084,3.746048,3.373122,13.931095,13.190062,27.121157,10.105947,17.015211,06180-1-1
4,5,urban_cluster,"POLYGON ((-79.51707 -5.59199, -79.51639 -5.583...",per,8.604389,8.442062,8.575487,8.116899,6.318737,5.689696,23.498613,22.248657,45.747271,17.046451,28.700820,02559-1-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35197,38677,urban_cluster,"POLYGON ((-43.98357 -6.43489, -43.98314 -6.426...",bra,268.545807,249.478012,257.162964,273.883209,264.637146,245.462540,790.345917,768.823761,1559.169678,518.023819,1041.145859,1016922
35198,38678,urban_cluster,"POLYGON ((-43.98314 -6.42678, -43.98271 -6.418...",bra,133.571472,124.087372,127.909767,136.226227,131.627350,122.090118,393.108589,382.403717,775.512306,257.658844,517.853462,1016923
35199,38679,urban_cluster,"POLYGON ((-43.97356 -6.43489, -43.97312 -6.426...",bra,51.492882,47.836681,49.310246,52.516315,50.743408,47.066730,151.546535,147.419727,298.966263,99.329563,199.636700,1016924
35200,38680,urban_cluster,"POLYGON ((-43.97312 -6.42678, -43.97269 -6.418...",bra,15.116997,14.043631,14.476233,15.417451,14.896971,13.817592,44.490200,43.278673,87.768873,29.160627,58.608246,1016925


In [35]:
gdf_cells_access_concat["polygon_id"].head()

0    02595-2-1
1    08801-1-1
2    06399-1-1
3    08147-1-1
4    05175-1-1
Name: polygon_id, dtype: object

In [36]:
gdf_cells_access_concat["polygon_id"].tail()

35197    03476-1-1
35198    02642-1-1
35199    02642-1-1
35200    02642-1-1
35201    02642-1-1
Name: polygon_id, dtype: object

In [37]:
gdf_cells_pop_concat["polygon_id"].head()

0    02559-1-1
1    07918-1-1
2    06180-1-1
3    06180-1-1
4    02559-1-1
Name: polygon_id, dtype: object

In [38]:
gdf_cells_pop_concat["polygon_id"].tail()

35197    1016922
35198    1016923
35199    1016924
35200    1016925
35201    1016926
Name: polygon_id, dtype: object

In [39]:
gdf_cells_access_concat["cell_id"] = gdf_cells_access_concat["cell_id"].astype("int32")
gdf_cells_pop_concat["cell_id"] = gdf_cells_pop_concat["cell_id"].astype("int32")

In [40]:
# Combine the two datasets using the index_col
gdf_combined = gdf_cells_access_concat[index_col + access_cols].merge(
    gdf_cells_pop_concat[index_col + common_cols + pop_cols],
    on=index_col,
    suffixes=("_access", "_pop"),
)

# Display the combined dataset
gdf_combined.head()

,cell_id,lat,lon,nearest_primary_schools_ix,distance_to_nearest_primary_schools,duration_to_nearest_primary_schools,duration_to_nearest_primary_schools_label,unique_id,lat_primary_school,lon_primary_school,...,pop_2020_f_5,pop_2020_m_10,pop_2020_f_10,pop_2020_m_15,pop_2020_f_15,pop_m,pop_f,pop_total,pop_primary_school_age,pop_secondary_school_age
0,13887,-10.947963,-69.576016,7622,1.6730,20.075000,15-30,boliviaID0013532,-10.947500,-69.564774,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,13956,-11.143462,-69.221002,7618,1.0347,12.415000,0-15,boliviaID0013394,-11.134600,-69.215324,...,0.034820,0.035746,0.032994,0.036197,0.029092,0.108391,0.096905,0.205296,0.071269,0.134028
2,13993,-11.281971,-69.069813,7589,0.2030,2.436667,0-15,boliviaID0013337,-11.280300,-69.069313,...,0.688169,0.706471,0.652080,0.715383,0.574967,2.142228,1.915215,4.057443,1.408542,2.648901
3,13998,-11.184197,-69.025069,7602,0.5786,6.943333,0-15,boliviaID0013370,-11.185100,-69.028412,...,0.465295,0.477670,0.440894,0.483696,0.388756,1.448436,1.294945,2.743381,0.952365,1.791016
4,14081,-11.877038,-68.988741,7537,0.5990,7.186667,0-15,boliviaID0013188,-11.879609,-68.986047,...,0.405146,0.415921,0.383899,0.421168,0.338501,1.261195,1.127546,2.388741,0.829252,1.559489


In [53]:
celdas_original = gpd.read_file(
    "Asentamientos humanos 2/Polígonos/Nuevos/Educación/CELDAS.gpkg"
)
celdas_original.shape

(37673, 14)

In [59]:
len(celdas_original.polygon_id.unique())

11096

In [ ]:
celdas_original["pop_2020"].sum()

np.float64(333804798.28630114)

In [57]:
celdas_original.drop_duplicates(subset="polygon_id")["pop_2020"].sum()

np.float64(38028451.20354667)

In [58]:
celdas_original.columns

Index(['cell_id', 'smod', 'pop_1975', 'pop_1980', 'pop_1985', 'pop_1990',
       'pop_1995', 'pop_2000', 'pop_2005', 'pop_2010', 'pop_2015', 'pop_2020',
       'polygon_id', 'geometry'],
      dtype='object')

In [43]:
# 333 804 798 <--- celdas originales
# 211.9M in 2020 segun el IBGE

In [44]:
gdf_combined["polygon_id"] = gdf_combined["polygon_id"].astype(str)

In [45]:
gdf_combined = gpd.GeoDataFrame(gdf_combined)

In [46]:
gdf_combined.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [47]:
gdf_combined.to_parquet("outputs/celdas_pop_distance_complete_v2.parquet", index=False)

In [48]:
gdf_combined.shape[0], gdf_cells_access_concat.shape[0], gdf_cells_pop_concat.shape[0]

(35202, 35202, 35202)

In [49]:
gdf_combined.columns

Index(['cell_id', 'lat', 'lon', 'nearest_primary_schools_ix',
       'distance_to_nearest_primary_schools',
       'duration_to_nearest_primary_schools',
       'duration_to_nearest_primary_schools_label', 'unique_id',
       'lat_primary_school', 'lon_primary_school',
       'nearest_secondary_schools_ix', 'distance_to_nearest_secondary_schools',
       'duration_to_nearest_secondary_schools',
       'duration_to_nearest_secondary_schools_label',
       'unique_id_secondary_school', 'lat_secondary_school',
       'lon_secondary_school', 'smod', 'polygon_id', 'geometry', 'country',
       'pop_2020_m_5', 'pop_2020_f_5', 'pop_2020_m_10', 'pop_2020_f_10',
       'pop_2020_m_15', 'pop_2020_f_15', 'pop_m', 'pop_f', 'pop_total',
       'pop_primary_school_age', 'pop_secondary_school_age'],
      dtype='object')

In [50]:
gdf_combined = gpd.GeoDataFrame(
    gdf_combined, geometry=gdf_combined.geometry, crs=gdf_cells_access.crs
)

In [51]:
gdf_combined["polygon_id"] = gdf_combined["polygon_id"].astype(str)

In [52]:
gdf_combined.to_file(
    "outputs/celdas_pop_distance_complete_v2.geojson", driver="GeoJSON", index=False
)

In [ ]:
gdf_combined.to_parquet("outputs/celdas_pop_distance_complete_v2.parquet", index=False)

In [ ]:
gdf_combined_stats = gdf_combined[
    [
        "cell_id",
        "smod",
        "country",
        "pop_total",
        "pop_primary_school_age",
        "pop_secondary_school_age",
        "distance_to_nearest_primary_schools",
        "duration_to_nearest_primary_schools",
        "duration_to_nearest_primary_schools_label",
        "distance_to_nearest_secondary_schools",
        "duration_to_nearest_secondary_schools",
        "duration_to_nearest_secondary_schools_label",
    ]
]

In [ ]:
gdf_combined_stats.head().to_clipboard()

In [ ]:
# Group by "country", "smod", and "duration_to_nearest_primary_schools_label" and calculate the sum of "pop_primary_school_age"
result = gdf_combined_stats.groupby(
    ["country", "smod", "duration_to_nearest_primary_schools_label"], as_index=False
)["pop_primary_school_age"].sum()

# Display the result
result

In [ ]:
# Save to an excel file
result.to_excel("outputs/pop_primary_school_age_2025_05_19.xlsx", index=False)

In [ ]:
# Group by "country", "smod", and "duration_to_nearest_schools_label" and calculate the sum of "pop_secondary_school_age"
result_secondary = gdf_combined_stats.groupby(
    ["country", "smod", "duration_to_nearest_secondary_schools_label"], as_index=False
)["pop_secondary_school_age"].sum()

# Display the result
result_secondary

In [ ]:
# Save to an excel file
result_secondary.to_excel(
    "outputs/pop_secondary_school_age_2025_05_19.xlsx", index=False
)

In [ ]:
result.head().to_clipboard()

In [ ]:
# Import required libraries
import pandas as pd
import seaborn as sns

In [ ]:
# Create a bar plot with seaborn
plt.figure(figsize=(8, 6))
sns.barplot(
    data=result_secondary,
    x="duration_to_nearest_secondary_schools_label",
    y="pop_secondary_school_age",
    hue="smod",
    errorbar=None,
)
plt.title("Population in Secondary School Age by Travel Time")
plt.xlabel("Travel Time to Nearest School (minutes)")
plt.ylabel("Population (Secondary School Age)")
plt.tight_layout()
plt.show()

In [ ]:
result_secondary

In [ ]:
for col in result_secondary.country.unique():
    # Create a bar plot with seaborn
    plt.figure(figsize=(8, 6))
    sns.barplot(
        data=result_secondary[result_secondary["country"] == col],
        x="duration_to_nearest_secondary_schools_label",
        y="pop_secondary_school_age",
        hue="smod",
        errorbar=None,
    )
    plt.title(f"{col}: Population in Secondary School Age by Travel Time")
    plt.xlabel("Travel Time to Nearest School (minutes)")
    plt.ylabel("Population (Secondary School Age)")
    plt.tight_layout()
    plt.savefig(f"outputs/{col}_secondary_school_access_plot.png")
    plt.show()

In [ ]:
result

In [ ]:
# (optional) set a clean style
sns.set(style="whitegrid", context="talk")

In [ ]:
# make a working df
df = gdf_combined_stats.copy()

plt.figure(figsize=(8, 6))
sns.scatterplot(
    x="duration_to_nearest_primary_schools",
    y="pop_primary_school_age",
    hue="country",
    palette="tab10",
    alpha=0.1,
    data=df,
)
plt.xlabel("Travel time to nearest primary school (min)")
plt.ylabel("Primary - school - age population")
plt.title("Primary - age children vs. travel time to primary school")
plt.legend(title="Country", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(
    x="duration_to_nearest_secondary_schools",
    y="pop_secondary_school_age",
    hue="country",
    palette="tab10",
    alpha=0.1,
    data=df,
)
plt.xlabel("Travel time to nearest secondary school (min)")
plt.ylabel("Secondary - school - age population")
plt.title("Secondary - age children vs. travel time to secondary school")
plt.legend(title="Country", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()

In [ ]:
# aggregate by country
country_stats = (
    df.groupby("country")
    .agg(
        total_primary_pop=("pop_primary_school_age", "sum"),
        mean_primary_time=("duration_to_nearest_primary_schools", "mean"),
        total_secondary_pop=("pop_secondary_school_age", "sum"),
        mean_secondary_time=("duration_to_nearest_secondary_schools", "mean"),
    )
    .reset_index()
)

fig, ax = plt.subplots(2, 1, figsize=(10, 12), sharex=True, sharey=True)

# primary
sns.scatterplot(
    x="mean_primary_time",
    y="total_primary_pop",
    size="total_primary_pop",
    sizes=(100, 2000),
    hue="country",
    data=country_stats,
    legend=False,
    alpha=0.5,
    ax=ax[0],
)
for _, row in country_stats.iterrows():
    ax[0].text(
        row.mean_primary_time,
        row.total_primary_pop,
        row.country.upper(),
        horizontalalignment="center",
        verticalalignment="center",
    )
ax[0].set_xlabel("Avg. primary - school travel time (min)")
ax[0].set_ylabel("Total primary - age population")
ax[0].set_title("Country - level: primary - age pop vs. avg. travel time")


# primary
sns.scatterplot(
    x="mean_secondary_time",
    y="total_secondary_pop",
    size="total_secondary_pop",
    sizes=(100, 2000),
    hue="country",
    data=country_stats,
    legend=False,
    alpha=0.5,
    ax=ax[1],
)
for _, row in country_stats.iterrows():
    ax[1].text(
        row.mean_secondary_time,
        row.total_secondary_pop,
        row.country.upper(),
        horizontalalignment="center",
        verticalalignment="center",
    )
ax[1].set_xlabel("Avg. secondary - school travel time (min)")
ax[1].set_ylabel("Total secondary - age population")
ax[1].set_title("Country - level: secondary - age pop vs. avg. travel time")

plt.tight_layout()
plt.show()

In [ ]:
smod_stats = (
    df.groupby("smod")
    .agg(
        total_primary_pop=("pop_primary_school_age", "sum"),
        mean_primary_time=("duration_to_nearest_primary_schools", "mean"),
        total_secondary_pop=("pop_secondary_school_age", "sum"),
        mean_secondary_time=("duration_to_nearest_secondary_schools", "mean"),
    )
    .sort_values("mean_primary_time", ascending=False)
    .reset_index()
)

# Primary
fig, ax = plt.subplots(1, 2, figsize=(14, 6), sharex=True)
sns.barplot(
    x="mean_primary_time", y="smod", data=smod_stats, ax=ax[0], order=smod_stats.smod
)
ax[0].set_title("Avg. primary travel time by settlement type")
ax[0].set_xlabel("Time (min)")
ax[0].set_ylabel("Settlement type")

# Secondary
sns.barplot(
    x="mean_secondary_time",
    y="smod",
    data=smod_stats.sort_values("mean_secondary_time", ascending=False),
    ax=ax[1],
)
ax[1].set_title("Avg. secondary travel time by settlement type")
ax[1].set_xlabel("Time (min)")
ax[1].set_ylabel("")
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as mcolors

In [ ]:
# Define duration bin order
duration_order = (
    gdf_combined_stats["duration_to_nearest_primary_schools_label"].dropna().unique()
)

In [ ]:
duration_order.to_list()

In [ ]:
cmap = cm.get_cmap("viridis", len(duration_order.to_list()))

In [ ]:
duration_colors = {
    label: mcolors.rgb2hex(cmap(i)) for i, label in enumerate(duration_order)
}
duration_colors

In [ ]:
# Cell: country‐level distribution of primary‐age pop by travel‐time label

# pivot to get total primary‐age pop by country × label
pop_label_country = (
    gdf_combined_stats.groupby(
        ["country", "duration_to_nearest_primary_schools_label"]
    )["pop_primary_school_age"]
    .sum()
    .unstack(fill_value=0)
)

# convert to fractions (so bars sum to 1)
pop_label_country_pct = pop_label_country.div(pop_label_country.sum(axis=1), axis=0)

# plot
ax = pop_label_country_pct.plot(
    kind="bar",
    stacked=True,
    figsize=(10, 6),
    color=[duration_colors[label] for label in duration_order],
    width=0.8,
)
ax.set_ylabel("Percentage of primary age population")
ax.set_xlabel("Country")
ax.set_title("Primary age population by travel time bins (primary school)")
ax.legend(title="Travel time (min)", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()

In [ ]:
# Cell: settlement‐type distribution of primary‐age pop by travel‐time label

pop_label_smod = (
    gdf_combined_stats.groupby(["smod", "duration_to_nearest_primary_schools_label"])[
        "pop_primary_school_age"
    ]
    .sum()
    .unstack(fill_value=0)
)
pop_label_smod_pct = pop_label_smod.div(pop_label_smod.sum(axis=1), axis=0)

ax = pop_label_smod_pct.plot(
    kind="bar",
    stacked=True,
    figsize=(10, 6),
    color=[duration_colors[label] for label in duration_order],
    width=0.8,
    rot=0,
)
ax.set_ylabel("Percentage of primary age population")
ax.set_xlabel("Settlement type")
ax.set_title("Primary age population by travel time bins (primary school)")
ax.legend(title="Travel time (min)", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()

In [ ]:
# Cell: country‐level distribution of secondary‐age pop by travel‐time label

# pivot to get total secondary‐age pop by country × label
pop_label_country_sec = (
    gdf_combined_stats.groupby(
        ["country", "duration_to_nearest_secondary_schools_label"]
    )["pop_secondary_school_age"]
    .sum()
    .unstack(fill_value=0)
)

# convert to fractions (so bars sum to 1)
pop_label_country_pct_sec = pop_label_country_sec.div(
    pop_label_country_sec.sum(axis=1), axis=0
)

# plot
ax = pop_label_country_pct_sec.plot(
    kind="bar",
    stacked=True,
    figsize=(10, 6),
    color=[duration_colors[label] for label in duration_order],
    width=0.8,
)
ax.set_ylabel("Percentage of secondary age population")
ax.set_xlabel("Country")
ax.set_title("secondary age population by travel time bins (secondary school)")
ax.legend(title="Travel time (min)", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()

In [ ]:
# Cell: settlement‐type distribution of secondary‐age pop by travel‐time label

pop_label_smod_sec = (
    gdf_combined_stats.groupby(["smod", "duration_to_nearest_secondary_schools_label"])[
        "pop_secondary_school_age"
    ]
    .sum()
    .unstack(fill_value=0)
)
pop_label_smod_pct_sec = pop_label_smod_sec.div(pop_label_smod_sec.sum(axis=1), axis=0)

ax = pop_label_smod_pct_sec.plot(
    kind="bar",
    stacked=True,
    figsize=(10, 6),
    color=[duration_colors[label] for label in duration_order],
    width=0.8,
    rot=0,
)
ax.set_ylabel("Percentage of secondary age population")
ax.set_xlabel("Settlement type")
ax.set_title("secondary age population by travel time bins (secondary school)")
ax.legend(title="Travel time (min)", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np

# Create a copy with relevant columns
df_lorenz = (
    gdf_combined_stats[
        ["duration_to_nearest_primary_schools", "pop_primary_school_age"]
    ]
    .dropna()
    .copy()
)

# Remove zero-pop cells
df_lorenz = df_lorenz[df_lorenz.pop_primary_school_age > 0]

# Sort by duration
df_lorenz.sort_values("duration_to_nearest_primary_schools", inplace=True)

# Compute cumulative population and cumulative share
df_lorenz["cum_pop"] = df_lorenz["pop_primary_school_age"].cumsum()
df_lorenz["cum_pop_share"] = (
    df_lorenz["cum_pop"] / df_lorenz["pop_primary_school_age"].sum()
)

# Weight durations by population (population-weighted Lorenz curve)
df_lorenz["duration_weighted"] = (
    df_lorenz["duration_to_nearest_primary_schools"]
    * df_lorenz["pop_primary_school_age"]
)
df_lorenz["cum_duration"] = df_lorenz["duration_weighted"].cumsum()
df_lorenz["cum_duration_share"] = (
    df_lorenz["cum_duration"] / df_lorenz["duration_weighted"].sum()
)

# Create a copy with relevant columns
df_lorenz_sec = (
    gdf_combined_stats[
        ["duration_to_nearest_secondary_schools", "pop_secondary_school_age"]
    ]
    .dropna()
    .copy()
)

# Remove zero-pop cells
df_lorenz_sec = df_lorenz_sec[df_lorenz_sec.pop_secondary_school_age > 0]

# Sort by duration
df_lorenz_sec.sort_values("duration_to_nearest_secondary_schools", inplace=True)

# Compute cumulative population and cumulative share
df_lorenz_sec["cum_pop"] = df_lorenz_sec["pop_secondary_school_age"].cumsum()
df_lorenz_sec["cum_pop_share"] = (
    df_lorenz_sec["cum_pop"] / df_lorenz_sec["pop_secondary_school_age"].sum()
)

# Weight durations by population (population-weighted Lorenz curve)
df_lorenz_sec["duration_weighted"] = (
    df_lorenz_sec["duration_to_nearest_secondary_schools"]
    * df_lorenz_sec["pop_secondary_school_age"]
)
df_lorenz_sec["cum_duration"] = df_lorenz_sec["duration_weighted"].cumsum()
df_lorenz_sec["cum_duration_share"] = (
    df_lorenz_sec["cum_duration"] / df_lorenz_sec["duration_weighted"].sum()
)

# Plot both Lorenz curves on the same plot
plt.figure(figsize=(8, 6))
plt.plot(
    df_lorenz["cum_pop_share"],
    df_lorenz["cum_duration_share"],
    label="Primary school age",
    color="blue",
    alpha=0.5,
)
plt.plot(
    df_lorenz_sec["cum_pop_share"],
    df_lorenz_sec["cum_duration_share"],
    label="Secondary school age",
    color="green",
    alpha=0.5,
)
plt.plot([0, 1], [0, 1], linestyle="--", color="gray", label="Line of equality")
plt.title("Lorenz Curve: Inequality in Access to Schools")
plt.xlabel("Cumulative share of population")
plt.ylabel("Cumulative share of total travel-time burden")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(
    data=gdf_combined_stats[gdf_combined_stats["pop_primary_school_age"] > 0],
    x="country",
    y="duration_to_nearest_primary_schools",
    color="skyblue",
    showfliers=False,
    width=0.4,
    position=1,
    boxprops=dict(alpha=0.7),
    linewidth=1.5,
    dodge=True,
    label="Primary",
)
sns.boxplot(
    data=gdf_combined_stats[gdf_combined_stats["pop_secondary_school_age"] > 0],
    x="country",
    y="duration_to_nearest_secondary_schools",
    color="orange",
    showfliers=False,
    width=0.4,
    position=0,
    boxprops=dict(alpha=0.7),
    linewidth=1.5,
    dodge=True,
    label="Secondary",
)
plt.ylabel("Travel time to school (min)")
plt.title("Distribution of Travel Times to Schools by Country")
plt.legend(
    handles=[
        plt.Line2D([0], [0], color="skyblue", lw=8, label="Primary"),
        plt.Line2D([0], [0], color="orange", lw=8, label="Secondary"),
    ]
)
plt.tight_layout()
plt.show()

In [ ]:
# Summary stats for Ecuador
ecuador = gdf_combined_stats[gdf_combined_stats["country"] == "ecu"]
print("Number of rows for Ecuador:", len(ecuador))
print("Non-zero population cells:", (ecuador["pop_primary_school_age"] > 0).sum())
print(
    "Non-null durations:",
    ecuador["duration_to_nearest_primary_schools"].notnull().sum(),
)
print(
    "Unique duration values:", ecuador["duration_to_nearest_primary_schools"].unique()
)

In [ ]:
gdf_combined_stats[gdf_combined_stats["country"] == "ecu"][
    ["pop_primary_school_age", "duration_to_nearest_primary_schools"]
].describe()

In [ ]:
sns.boxplot(
    data=gdf_combined_stats[
        (gdf_combined_stats["pop_primary_school_age"] > 0)
        & (gdf_combined_stats["duration_to_nearest_primary_schools"] > 0)
    ],
    x="country",
    y="duration_to_nearest_primary_schools",
    showfliers=False,  # hide outliers for readability,
)
plt.ylabel("Travel time to primary school (min)")
plt.title("Travel Times to Primary Schools by Country")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(
    data=gdf_combined_stats[gdf_combined_stats["pop_primary_school_age"] > 0],
    x="smod",
    y="duration_to_nearest_primary_schools",
    order=["urban_center", "urban_cluster", "rural"],
    showfliers=False,
)
plt.ylabel("Travel time to primary school (min)")
plt.title("Travel Time Distribution by Settlement Type")
plt.tight_layout()
plt.show()

In [ ]:
duration_order

In [ ]:
# Make sure all needed columns are included in the copy
df_facet = gdf_combined_stats[
    [
        "country",
        "smod",
        "pop_primary_school_age",
        "pop_secondary_school_age",
        "duration_to_nearest_primary_schools_label",
        "duration_to_nearest_secondary_schools_label",
    ]
].copy()

# Melt the population columns
df_melted = pd.melt(
    df_facet,
    id_vars=[
        "country",
        "smod",
        "duration_to_nearest_primary_schools_label",
        "duration_to_nearest_secondary_schools_label",
    ],
    value_vars=["pop_primary_school_age", "pop_secondary_school_age"],
    var_name="school_level",
    value_name="pop_school_age",
)

# Assign matching duration labels
df_melted["duration_label"] = df_melted.apply(
    lambda row: (
        row["duration_to_nearest_primary_schools_label"]
        if row["school_level"] == "pop_primary_school_age"
        else row["duration_to_nearest_secondary_schools_label"]
    ),
    axis=1,
)

# Clean up school level name
df_melted["school_level"] = df_melted["school_level"].map(
    {"pop_primary_school_age": "Primary", "pop_secondary_school_age": "Secondary"}
)

# Filter out zero-pop rows
df_melted = df_melted[df_melted["pop_school_age"] > 0]

# Facet by country
g = sns.catplot(
    data=df_melted,
    kind="bar",
    x="duration_label",
    y="pop_school_age",
    hue="school_level",
    hue_order=["Primary", "Secondary"],
    col="country",
    col_wrap=2,
    order=duration_order,
    height=4,
    aspect=1,
)

g.set_axis_labels("Travel time bin (min)", "School-age population")
g.set_titles("Country: {col_name}")
g._legend.set_title("School level")
g._legend.set_loc("lower right")
plt.tight_layout()
plt.show()